![Image of Toronto Skyline](https://imgc.allpostersimages.com/img/print/u-g-PTD3410.jpg?w=550&h=550&p=0)
# Clustering of Neighborhoods in Toronto
**_image credit: imgc.allpostersimages.com_**

### Importing required libraries

In [3]:
import requests # library to handle requests
import pandas as pd # library for data analysis
import numpy as np # library to handle data in a vectorized manner
import json # library to handle JSON files
!pip install geopy 
# module to convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
# transforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
# install Folium
!pip install folium
import folium 
# install beautiful soup 4
!pip install bs4
print('All necessary libraries are installed..ready to roll')

     |████████████████████████████████| 102kB 5.4MB/s ta 0:00:011
  Stored in directory: /home/dsxuser/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4
All necessary libraries are installed..ready to roll


### Using the requests module to get the HTML of the wikipedia page

In [49]:
wiki_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
result_txt = requests.get(wiki_url).text
result_txt[:1000]

'\n<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>List of postal codes of Canada: M - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XrRJvgpAEKcAAI@-EI0AAAAQ","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":955414546,"wgRevisionId":955414546,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related l

### Scrapping the wikipedia page using Beautiful Soup

In [50]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(result_txt,'html.parser')
print(soup.prettify()[:1000])

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XrRJvgpAEKcAAI@-EI0AAAAQ","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":955414546,"wgRevisionId":955414546,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toronto","Ontario

### Creating Pandas DataFrame from collected data

In [51]:
# parsing the table part of the response 
postal_table = soup.find('table',attrs={'class':'wikitable sortable'})
# Let's check the headers in postal_table
table_headers = postal_table.find_all('th')
# Let's check the rows in HTML postal_table
table_rows = postal_table.find_all('tr')
print("Total number of rows in the table:",len(table_rows))
# Let's check the elements in HTML postal_table
table_elements = postal_table.find_all('td')
# Initiate an array row_values
row_values = []
# Fill the array row_values
for rows in table_rows:
     data = rows.find_all('td') # finding the elements in each row
     values = [rows.text.strip() for rows in data if rows.text.strip()]
     if values:
        row_values.append(values) # Adding elements
# Initiate column names for the dataframe
column_names = ["PostalCode", "Borough", "Neighborhood"]
# Create the initial pandas dataframe 
toronto_df = pd.DataFrame(row_values, columns=column_names)
# Let's check the shape of the dataframe
print("Shape of the Postal Code DataFrame:",toronto_df.shape)
toronto_df.head()

Total number of rows in the table: 181
Shape of the Postal Code DataFrame: (180, 3)


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,None
1,M2A,Not assigned,None
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Cleaning the DataFrame

In [52]:
# Only process cells that have an assigned burough, ignore cells where burough is "Not Assigned" 
toronto_df['Borough'].replace('Not assigned', np.nan, inplace=True)
toronto_df.dropna(subset=['Borough'], inplace=True)
toronto_df.reset_index(inplace=True, drop=True)
print("Shape of cleaned Postal Code DataFrame:", toronto_df.shape)
toronto_df.head()

Shape of cleaned Postal Code DataFrame: (103, 3)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [53]:
# If a cell has a Borough, but a "Not assigned" Neighborhood, then the Neighborhood will be same as Borough
toronto_df['Neighborhood'] = toronto_df.apply(
lambda row:
row['Borough']if row['Neighborhood']=='Not assigned'
else row['Neighborhood'], axis=1)
print("Shape of cleaned Postal Code DataFrame:",toronto_df.shape)
toronto_df.head()

Shape of cleaned Postal Code DataFrame: (103, 3)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [54]:
# Let's count the number of cases where Neighborhood is "Not assigned"
toronto_df.loc[toronto_df.Neighborhood == 'Not assigned','Neighborhood'].count()

0

In [55]:
# If more than one neighborhood exists in one postal code area, let's combine them in one row
df_toronto = toronto_df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(lambda tags:','.join(tags)).reset_index(drop=False)
df_toronto.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [56]:
# Let's see the number of rows in the Dataframe using .shape method
print("\033[1m Final Postal Code Dataframe shape:",df_toronto.shape)

 Final Postal Code Dataframe shape: (103, 3)


### Since Geocoder package is unreliable, a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [57]:
geo_csv = 'https://cocl.us/Geospatial_data'
df_geo = pd.read_csv(geo_csv)
print("Shape of Geospatial DataFrame:",df_geo.shape)
df_geo.head()

Shape of Geospatial DataFrame: (103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


**_The shape of the Geospatial dataframe is same as our final Postal Code dataframe_**

In [58]:
# Rename the first column in df_geo to prevent a new column while merging
df_geo.rename(columns={"Postal Code":"PostalCode"},inplace = True)
# Merging two dataframes on basis of PostalCode
toronto_final = pd.merge(df_toronto,df_geo,left_on='PostalCode',right_on='PostalCode',how='left')
toronto_final.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [59]:
print('Our combined dataframe has {} unique boroughs and {} neighborhoods.'.format(len(toronto_final['Borough'].unique()),
        toronto_final.shape[0]))

Our combined dataframe has 10 unique boroughs and 103 neighborhoods.


## Explore and cluster the neighborhoods of boroughs that contain the word Toronto

In [60]:
# Creating a truncated dataframe with boroughs that contain the word Toronto
toronto_data = toronto_final[toronto_final['Borough'].str.contains('Toronto')].reset_index(drop=True)
print("Shape of dataframe of Toronto boroughs:",toronto_data.shape)
toronto_data.head()

Shape of dataframe of Toronto boroughs: (39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [61]:
print('Toronto neighborhood dataframe has {} unique boroughs and {} neighborhoods.'.format(len(toronto_data['Borough'].unique()),
        toronto_data.shape[0]))

Toronto neighborhood dataframe has 4 unique boroughs and 39 neighborhoods.


In [62]:
# List of unique values in "Borough" column
toronto_data.Borough.unique()

array(['East Toronto', 'Central Toronto', 'Downtown Toronto',
       'West Toronto'], dtype=object)

**_The unique boroughs are East Toronto, Central Toronto, Downtown Toronto and West Toronto_**

In [63]:
# Let's check the number of postal codes for each unique borough
Postal_Codes = toronto_data.groupby("Borough")['PostalCode'].count()
Postal_Codes.head()

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
West Toronto         6
Name: PostalCode, dtype: int64

### Using geopy let's get the longitude and latitude of Toronto

In [64]:
address = 'Toronto, Ontario, Canada'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [46]:
# Create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)
# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=6,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
# Save map 
map_toronto.save("Toronto_Map.html")
# Display map   
HTML(map_toronto._repr_html_())

### Define Foursquare Credentials and Version

In [65]:
# The code was removed by Watson Studio for sharing.

**_Let's explore the first neighborhood in our dataframe_**

In [66]:
print('The first neighborhood in our dataframe is:',toronto_data.loc[0, 'Neighborhood'])

The first neighborhood in our dataframe is: The Beaches


**_Let's get the latitude and longitude of the first neighborhood_**

In [67]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


**_Let's get the top 100 venues that are in The Beaches within a radius of 500 meters_**

In [68]:
# Define the radius and limit
radius = 500
LIMIT = 100
# Create the url
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
# Send the GET request and examine results
results = requests.get(url).json()
# Print results
results

{'meta': {'code': 200, 'requestId': '5ebd70ffc94979001b2fb0ab'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 71,
  'suggestedBounds': {'ne': {'lat': 43.6579817045, 'lng': -79.37772678059432},
   'sw': {'lat': 43.6489816955, 'lng': -79.39014261940568}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5227bb01498e17bf485e6202',
       'name': 'Downtown Toronto',
       'location': {'lat': 43.65323167517444,
        'lng': -79.38529600606677,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.65323167517444,
          'lng'

**_Let's define a function: get_category_type that extracts the category of the venue_**

In [70]:
# function get_category_type
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [71]:
from pandas.io.json import json_normalize
# Create a dataframe venues
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
print("Shape of nearby venues of The Beach borough:",nearby_venues.shape)
nearby_venues.head()

Shape of nearby venues of The Beach borough: (71, 4)


,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Nathan Phillips Square,Plaza,43.652270,-79.383516
2,Indigo,Bookstore,43.653515,-79.380696
3,Chatime 日出茶太,Bubble Tea Shop,43.655542,-79.384684
4,Textile Museum of Canada,Art Museum,43.654396,-79.386500


## Let's explore Neighborhoods in Toronto

**_We have already explored the first neighborhood, let's create a function that would repeat for all neighborhoods_**

In [72]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

**_Let's use the above function on each neighborhood and create a new dataframe_**

In [73]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High Park, The Junction South
Parkdale, Ron

In [74]:
print("Shape of nearby venues dataframe:",toronto_venues.shape)
toronto_venues.head()

Shape of nearby venues dataframe: (1614, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Glen Stewart Park,43.675278,-79.294647,Park
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


**_Let's check how many venues were returned for each neighborhood_**

In [75]:
toronto_venues.groupby('Neighborhood')['Venue'].count().sort_values(ascending=False)

Neighborhood
Toronto Dominion Centre, Design Exchange                                                                      100
Commerce Court, Victoria Hotel                                                                                100
First Canadian Place, Underground city                                                                        100
Garden District, Ryerson                                                                                      100
Harbourfront East, Union Station, Toronto Islands                                                             100
Stn A PO Boxes                                                                                                 94
Richmond, Adelaide, King                                                                                       93
St. James Town                                                                                                 78
Church and Wellesley                                                       

**_5 neighborhoods returned 100 venues, while "Roselawn" neighborhood returned only 1 venue_**

**_Let's find out how many unique categories are present in all the returned venues_**

In [76]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 234 uniques categories.


## Analyze Each Neighborhood

In [77]:
# one hot encoding to create dummy columns
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']],prefix="",prefix_sep="")
# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

#### Let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [78]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
print("Shape of grouped neighborhood dataframe:",toronto_grouped.shape)
toronto_grouped.head()

Shape of grouped neighborhood dataframe: (39, 234)


,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Berczy Park,0.000000,0.0,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.017544,0.0,0.0,0.0,0.0,0.0,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.0,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,Business reply mail Processing Centre,0.000000,0.0,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0,0.0625,0.0625,0.125,0.125,0.125,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,Central Bay Street,0.015625,0.0,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.015625,0.0,0.0,0.0,0.0,0.0,0.0


#### Let's print each neighborhood along with the top 5 most common venues

In [79]:
# Number of top 5 most common venues
num_top_venues = 5
# Creating the list of top 5 most common venues
for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----") # Print the neighborhood in first line
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq'] # Column headings
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float) # Freq is cast as float
    temp = temp.round({'freq': 2}) # Rounded by two decimal points
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
          venue  freq
0   Coffee Shop  0.07
1  Cocktail Bar  0.05
2      Beer Bar  0.04
3          Café  0.04
4   Cheese Shop  0.04


----Brockton, Parkdale Village, Exhibition Place----
                venue  freq
0                Café  0.14
1         Coffee Shop  0.09
2      Breakfast Spot  0.09
3       Grocery Store  0.05
4  Italian Restaurant  0.05


----Business reply mail Processing Centre----
                  venue  freq
0                Garden  0.06
1         Auto Workshop  0.06
2            Skate Park  0.06
3  Fast Food Restaurant  0.06
4            Smoke Shop  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                 venue  freq
0       Airport Lounge  0.12
1      Airport Service  0.12
2     Airport Terminal  0.12
3             Boutique  0.06
4  Rental Car Location  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.17
1                Café

### Let's cast the above list into a pandas dataframe

**_We define a function "return_most_common_venues" to sort the venues in descending order_**

In [80]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

**_Using the above function we create the new dataframe and display the top 10 venues for each neighborhood_**

In [81]:
num_top_venues = 10
# These indicators will be used to write the 1st 3 column headers
indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']
for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)
# Display the sorted dataframe
print("Shape of grouped & sorted neighborhood dataframe:",neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head()

Shape of grouped & sorted neighborhood dataframe: (39, 11)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Beer Bar,Restaurant,Café,Seafood Restaurant,Bakery,Comfort Food Restaurant,Museum
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Grocery Store,Stadium,Burrito Place,Restaurant,Climbing Gym,Pet Store,Bakery
2,Business reply mail Processing Centre,Pizza Place,Recording Studio,Restaurant,Fast Food Restaurant,Light Rail Station,Farmers Market,Auto Workshop,Spa,Smoke Shop,Burrito Place
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Harbor / Marina,Sculpture Garden,Rental Car Location,Plane,Coffee Shop,Bar
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Ice Cream Shop,Burger Joint,Bar,Department Store,Salad Place,Japanese Restaurant


## Cluster Neighborhoods

### Run K-Means to create 3 distinct clusters

In [82]:
# set number of clusters
kclusters = 3
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:30] 

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 0, 2,
       2, 2, 2, 2, 0, 1, 2, 2], dtype=int32)

**_A final dataframe that includes Postal code, Borough, Neighborhood, Latitude, Longitude,
Cluster labels as well as the top 10 venues for each neighborhood_**

In [83]:
# Let's add clustering labels to the dataframe
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
# New Dataframe copied from toronto_data
toronto_merged = toronto_data
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
# Display the sorted dataframe
print("Shape of the final dataframe:",toronto_merged.shape)
toronto_merged.head() 

Shape of the final dataframe: (39, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Park,Health Food Store,Pub,Trail,Donut Shop,Doner Restaurant,Dog Run,Eastern European Restaurant,Cupcake Shop,Distribution Center
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,2,Greek Restaurant,Italian Restaurant,Coffee Shop,Restaurant,Ice Cream Shop,Furniture / Home Store,Fruit & Vegetable Store,Pub,Pizza Place,Lounge
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,2,Sandwich Place,Fast Food Restaurant,Food & Drink Shop,Brewery,Liquor Store,Restaurant,Burrito Place,Italian Restaurant,Intersection,Ice Cream Shop
3,M4M,East Toronto,Studio District,43.659526,-79.340923,2,Café,Coffee Shop,Gastropub,Bakery,Brewery,American Restaurant,Yoga Studio,Comfort Food Restaurant,Bookstore,Sandwich Place
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Park,Swim School,Bus Line,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run


## Visualizing the unique clusters

In [84]:
# Import Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=6,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.9).add_to(map_clusters)
# Save map
map_clusters.save("Toronto_Clustered_Map.html")
# Display map   
HTML(map_clusters._repr_html_())

## Examine the Unique Clusters

### Let's examine Cluster # 1

In [85]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Park,Health Food Store,Pub,Trail,Donut Shop,Doner Restaurant,Dog Run,Eastern European Restaurant,Cupcake Shop,Distribution Center
4,Central Toronto,0,Park,Swim School,Bus Line,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run
8,Central Toronto,0,Park,Restaurant,Playground,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
10,Downtown Toronto,0,Park,Playground,Trail,Cupcake Shop,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
23,Central Toronto,0,Park,Jewelry Store,Trail,Sushi Restaurant,Dance Studio,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run


### Let's examine Cluster # 2

In [86]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,1,Health & Beauty Service,Garden,Women's Store,Deli / Bodega,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant


### Let's examine Cluster # 3

In [87]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Toronto,2,Greek Restaurant,Italian Restaurant,Coffee Shop,Restaurant,Ice Cream Shop,Furniture / Home Store,Fruit & Vegetable Store,Pub,Pizza Place,Lounge
2,East Toronto,2,Sandwich Place,Fast Food Restaurant,Food & Drink Shop,Brewery,Liquor Store,Restaurant,Burrito Place,Italian Restaurant,Intersection,Ice Cream Shop
3,East Toronto,2,Café,Coffee Shop,Gastropub,Bakery,Brewery,American Restaurant,Yoga Studio,Comfort Food Restaurant,Bookstore,Sandwich Place
5,Central Toronto,2,Park,Gym,Breakfast Spot,Sandwich Place,Food & Drink Shop,Hotel,Department Store,Deli / Bodega,Eastern European Restaurant,Donut Shop
6,Central Toronto,2,Clothing Store,Coffee Shop,Grocery Store,Gift Shop,Seafood Restaurant,Salon / Barbershop,Restaurant,Pet Store,Park,Mexican Restaurant
7,Central Toronto,2,Pizza Place,Sandwich Place,Dessert Shop,Café,Coffee Shop,Gym,Sushi Restaurant,Italian Restaurant,Gas Station,Pharmacy
9,Central Toronto,2,Pub,Coffee Shop,Restaurant,Supermarket,Sushi Restaurant,Bank,Sports Bar,Fried Chicken Joint,Bagel Shop,Pizza Place
11,Downtown Toronto,2,Coffee Shop,Restaurant,Italian Restaurant,Pub,Bakery,Café,Pizza Place,Grocery Store,Pharmacy,Sandwich Place
12,Downtown Toronto,2,Japanese Restaurant,Sushi Restaurant,Coffee Shop,Restaurant,Yoga Studio,Hotel,Gay Bar,Café,Gastropub,Mediterranean Restaurant
13,Downtown Toronto,2,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Pub,Theater,Mexican Restaurant,Shoe Store,Restaurant


## Summary of clusters grouped by Cluster Labels & Borough

In [89]:
toronto_merged.groupby(["Cluster Labels","Borough"])['Cluster Labels'].count().sort_values(ascending=False)

Cluster Labels  Borough         
2               Downtown Toronto    18
                West Toronto         6
                Central Toronto      5
                East Toronto         4
0               Central Toronto      3
1               Central Toronto      1
0               East Toronto         1
                Downtown Toronto     1
Name: Cluster Labels, dtype: int64

## No of Boroughs returned for different k_clusters

| k_cluster | Cluster 1  | Cluster 2  | Cluster 3  | Cluster 4  | Cluster 5  |
|:--------- |:----------:|:----------:|:----------:|:----------:|-----------:|
|    5      |    35      |     1      |     1      |     1      |    1       |
|    4      |    1       |    35      |     1      |     2      |    NaN     |
|    3      |    5       |     1      |     33     |   NaN      |    NaN     |

## For k_clusters = 3, we segmented the neighborhood in following clusters
#### Cluster 1 : Park is the Most Common Venue
#### Cluster 2 : Health & Beauty Service is the Most Common Venue
#### Cluster 3 :Coffee shops/Restaurants is the Most Common Venue